# Tutorial: Batch Processing

This tutorial shows how to solve for solutions to different scenarios in a batch processing fashion. You can find all realted files in [this](https://gitlab.lrz.de/tum-cps/commonroad_io/tree/batch_processing) repository. The main components are:
* a configuration file (.yaml format),
* some helper functions,
* and the script below to call the planer function.

Before proceeding, you should make sure the configuration file is set correctly. Here are the explanation for each of the parameters in the configuration file:
* **input_path**: input directory of CommonRoad scenarios that you indend to solve.
* **output_path**: output directory of the solution files.
* **overwrite**: flag to determine whether the existing solution files should be overwritten.
* **timeout**: timeout time for your motion planner, unit in seconds
* **motion_planner_path**: directory where the module containing the function to execute your motion planner is located
* **motion_planner_module_name**: name of the module that contains the function to execute your motion planner
* **motion_planner_function_name**: name of the function that executes your motion planner
* **default**: parameters specified under this section will be applied to all scenarios. If you wish to specify a different paramter for specific scenarios, simply copy the section and replace 'default' with the id of your scenario.
* **vehicle_model**: model of the vehicle, valid values: **PM, KS, ST and MB**. Please refer to [Vehicle Models](https://gitlab.lrz.de/tum-cps/commonroad-vehicle-models/blob/master/vehicleModels_commonRoad.pdf) for more information.
* **vehicle_type**: type of the vehicle, valid values: **FORD_ESCORT, BMW_320i and VW_VANAGON**.
* **cost_function**: identifier of cost function. Please refer to [Cost Functions](https://gitlab.lrz.de/tum-cps/commonroad-cost-functions/blob/master/costFunctions_commonRoad.pdf) for more information.
* **planner_id**: id of the planner that is used to solve for solutions. in this example, 1 = Greedy Best First Search, 2 = A*, else = Your own planner.
* **planning_problem_idx**: planner problem index. for cooperative scenarios: 0, 1, 2, ..., otherwise: 0
* **max_tree_depth**: maximum permissible depth of the search tree

Note: the paths can be either **relative** to this script or **absolute**.

Simply run the following script to start batch processing. The results here is just exemplary, you will see different output as your configuration varies.

In [ ]:
# import helper functions
from helper_functions import *
from multiprocessing import Pool
import os

NUM_PROCESSES = 16

#source: https://stackoverflow.com/questions/6974695/python-process-pool-non-daemonic
pool = Pool(processes=8)
class NoDaemonProcess(multiprocessing.Process):
    @property
    def daemon(self):
        return False

    @daemon.setter
    def daemon(self, value):
        pass

class NoDaemonContext(type(multiprocessing.get_context())):
    Process = NoDaemonProcess

# We sub-class multiprocessing.pool.Pool instead of multiprocessing.Pool
# because the latter is only a wrapper function, not a proper class.
class MyPool(multiprocessing.pool.Pool):
    def __init__(self, *args, **kwargs):
        kwargs['context'] = NoDaemonContext()
        super(MyPool, self).__init__(*args, **kwargs)
        

# specify path to configuration file
path_file_config = "../configuration/batch_processing_config.yaml"

# load configuration file
configuration = load_configuration(path_file_config)

# get target function
function_target = get_target_function(configuration)

# get a list of scenario files
list_files_input_all = get_input_files(configuration)

solution_files = os.listdir('../../../solutions/')
list_files_input = []
for scenario in list_files_input_all:
    already_solved = False
    for solution in solution_files:
        if scenario.strip('.xml') in solution:
            already_solved = True
            break
    if not already_solved:
        list_files_input.append(scenario)

time_timeout = configuration['timeout']        
print(f'time_timeout: {time_timeout}s')
num_files = len(list_files_input)
print(f'It\'s going to take a maximum of {time_timeout*num_files/(NUM_PROCESSES * 3600):.2f}h')
print("Total number of files to be processed: {}".format(num_files))
print("Timeout setting: {} seconds\n".format(time_timeout))
count_processed = 0
number_solutions = 0

def compute_trajectory(file_scenario):
    # parse the scenario file
    result_parse = parse_scenario_file(configuration, file_scenario)
    # execute target function
    solution_trajectories = execute_target_function(function_target, result_parse, time_timeout)
    return (solution_trajectories, result_parse)

for i in range(int(len(list_files_input)/NUM_PROCESSES)):
    input_files = list_files_input[i*NUM_PROCESSES:(i+1)*NUM_PROCESSES]
    pool = MyPool(processes=NUM_PROCESSES)
    results = pool.map(compute_trajectory, input_files)

    for result in results:
        solution_trajectories = result[0]
        result_parse = result[1]
        save_solution(configuration, solution_trajectories, result_parse)
        if solution_trajectories != None:
            number_solutions += 1
    print(f'Solved {number_solutions}/{num_files}' )

time_timeout: 60s
It's going to take a maximum of 0.19h
Total number of files to be processed: 187
Timeout setting: 60 seconds

Reading motion primitives...
Reading motion primitives...
Reading motion primitives...
Reading motion primitives...
Reading motion primitives...
Reading motion primitives...
Reading motion primitives...
Reading motion primitives...
Reading motion primitives...
Reading motion primitives...
Reading motion primitives...
Reading motion primitives...
Reading motion primitives...
Reading motion primitives...
Reading motion primitives...
Reading motion primitives...
Automata created.
Number of loaded primitives: 3066
Planning..
Automata created.
Number of loaded primitives: 3066
Automata created.
Number of loaded primitives: 3066
Planning..
Planning..
Automata created.
Number of loaded primitives: 3066
Automata created.
Number of loaded primitives: 3066
Automata created.
Number of loaded primitives: 3066
Planning..
Automata created.
Number of loaded primitives: 3066


Automata created.
Number of loaded primitives: 3066
Planning..
Automata created.
Number of loaded primitives: 3066
Automata created.
Number of loaded primitives: 3066
Automata created.
Number of loaded primitives: 3066
Automata created.
Number of loaded primitives: 3066
Planning..
Automata created.
Planning..
Number of loaded primitives: 3066
Planning..
Automata created.
Number of loaded primitives: 3066
Planning..
Automata created.
Number of loaded primitives: 3066
Planning..
Planning..
Planning..
Automata created.
Number of loaded primitives: 3066
Automata created.
Number of loaded primitives: 3066
Automata created.
Number of loaded primitives: 3066
Planning..
Planning..
Planning..
Automata created.
Number of loaded primitives: 3066
Automata created.
Number of loaded primitives: 3066
Automata created.
Number of loaded primitives: 3066
Planning..
Planning..
Planning..
Automata created.
Number of loaded primitives: 3066
Planning..
Automata created.
Number of loaded primitives: 3066
Pla

Number of loaded primitives: 3066
Planning..
Planning..
Planning..
Automata created.
Number of loaded primitives: 3066
Automata created.
Number of loaded primitives: 3066
Automata created.
Number of loaded primitives: 3066
Automata created.
Number of loaded primitives: 3066
Planning..
Planning..
Planning..
Automata created.
Number of loaded primitives: 3066
Planning..
Planning..
Automata created.
Number of loaded primitives: 3066
Planning..
Planning..
<Trajectory/state_list>: all states must have the same attributes. Attributes of first state: ['position', 'orientation', 'velocity', 'steering_angle', 'acceleration', 'time_step'].
<TIMEOUT> Motion planner timeout.
<TIMEOUT> Motion planner timeout.
<TIMEOUT> Motion planner timeout.
<TIMEOUT> Motion planner timeout.
<TIMEOUT> Motion planner timeout.
<TIMEOUT> Motion planner timeout.
<TIMEOUT> Motion planner timeout.
<TIMEOUT> Motion planner timeout.
<TIMEOUT> Motion planner timeout.
<TIMEOUT> Motion planner timeout.
<TIMEOUT> Motion plann

<TIMEOUT> Motion planner timeout.
<TIMEOUT> Motion planner timeout.
<TIMEOUT> Motion planner timeout.
<TIMEOUT> Motion planner timeout.
<TIMEOUT> Motion planner timeout.
<TIMEOUT> Motion planner timeout.
<TIMEOUT> Motion planner timeout.
<TIMEOUT> Motion planner timeout.
<TIMEOUT> Motion planner timeout.
<TIMEOUT> Motion planner timeout.
<TIMEOUT> Motion planner timeout.
<FAILURE> Solution not Found
<FAILURE> Solution not Found
<FAILURE> Solution not Found
<FAILURE> Solution not Found
<SUCCESS> Solution file written.
<FAILURE> Solution not Found
<FAILURE> Solution not Found
<FAILURE> Solution not Found
<FAILURE> Solution not Found
<FAILURE> Solution not Found
<FAILURE> Solution not Found
<FAILURE> Solution not Found
<FAILURE> Solution not Found
<FAILURE> Solution not Found
<FAILURE> Solution not Found
<FAILURE> Solution not Found
Solved 2/187
Reading motion primitives...
Reading motion primitives...
Reading motion primitives...
Reading motion primitives...
Reading motion primitives...
